<a href="https://colab.research.google.com/github/michaelewilliams08/bi_portfolio_project/blob/master/311_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests

# --- Step 1: City Metadata ---
CITIES = [
    {"city": "New York",      "state": "NY", "state_fips": "36", "place_fips": "51000"},
    {"city": "Los Angeles",   "state": "CA", "state_fips": "06", "place_fips": "44000"},
    {"city": "Chicago",       "state": "IL", "state_fips": "17", "place_fips": "14000"},
    {"city": "Houston",       "state": "TX", "state_fips": "48", "place_fips": "35000"},
    {"city": "Miami",         "state": "FL", "state_fips": "12", "place_fips": "45000"},
]

# --- Step 2: Static Fallback Data (from ACS 2022 5-Year Estimates) ---
CENSUS_CACHE = {
    "New York":      {"population": 8335897, "median_income": 74694},
    "Los Angeles":   {"population": 3822238, "median_income": 69778},
    "Chicago":       {"population": 2665039, "median_income": 65781},
    "Houston":       {"population": 2302878, "median_income": 59408},
    "Miami":         {"population": 449514,  "median_income": 53975},
}

# --- Step 3: Robust Fetch Function ---
def fetch_city_census_data(city_row, use_cache=True):
    """
    Tries to fetch population & median income from US Census API.
    Falls back to local cache if API fails or use_cache=True.
    """
    if not use_cache:
        try:
            variables = ['B01003_001E', 'B19013_001E']
            url = (
                f'https://api.census.gov/data/2022/acs/acs5'
                f'?get={",".join(variables)}&for=place:{city_row["place_fips"]}&in=state:{city_row["state_fips"]}'
            )
            resp = requests.get(url, timeout=10)
            resp.raise_for_status()
            data = resp.json()[1]
            return {
                "population": int(data[0]),
                "median_income": int(data[1]),
            }
        except Exception as e:
            print(f"API failed for {city_row['city']}: {e}\nFalling back to local cache.")
    return CENSUS_CACHE[city_row['city']]

# --- Step 4: Build DataFrame ---
city_rows = []
for city in CITIES:
    data = fetch_city_census_data(city, use_cache=True)  # Always True for Colab reliability
    city_rows.append({
        "city": city["city"],
        "state": city["state"],
        **data
    })
df_census = pd.DataFrame(city_rows)

# --- Step 5: Display ---
df_census

,city,state,population,median_income
0,New York,NY,8335897,74694
1,Los Angeles,CA,3822238,69778
2,Chicago,IL,2665039,65781
3,Houston,TX,2302878,59408
4,Miami,FL,449514,53975


In [2]:
import pandas as pd
import requests

# --- City Metadata ---
CITIES = [
    {"city": "New York",      "state": "NY", "state_fips": "36", "place_fips": "51000"},
    {"city": "Los Angeles",   "state": "CA", "state_fips": "06", "place_fips": "44000"},
    {"city": "Chicago",       "state": "IL", "state_fips": "17", "place_fips": "14000"},
    {"city": "Houston",       "state": "TX", "state_fips": "48", "place_fips": "35000"},
    {"city": "Miami",         "state": "FL", "state_fips": "12", "place_fips": "45000"},
]

# --- Step 1: Census Data (ACS 2022 5-Year, https://data.census.gov/) ---
CENSUS_CACHE = {
    "New York":      {"population": 8335897, "median_income": 74694},
    "Los Angeles":   {"population": 3822238, "median_income": 69778},
    "Chicago":       {"population": 2665039, "median_income": 65781},
    "Houston":       {"population": 2302878, "median_income": 59408},
    "Miami":         {"population": 449514,  "median_income": 53975},
}

def fetch_city_census_data(city_row, use_cache=True):
    if not use_cache:
        try:
            variables = ['B01003_001E', 'B19013_001E']
            url = (
                f'https://api.census.gov/data/2022/acs/acs5'
                f'?get={",".join(variables)}&for=place:{city_row["place_fips"]}&in=state:{city_row["state_fips"]}'
            )
            resp = requests.get(url, timeout=10)
            resp.raise_for_status()
            data = resp.json()[1]
            return {
                "population": int(data[0]),
                "median_income": int(data[1]),
            }
        except Exception as e:
            print(f"API failed for {city_row['city']}: {e}\nFalling back to local cache.")
    return CENSUS_CACHE[city_row['city']]

# --- Step 2: Crime Data (FBI UCR 2022, https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/explorer) ---
# Values are violent crime rate per 100,000 people, 2022. Sources: FBI UCR, city reports.
CRIME_CACHE = {
    "New York":      {"violent_crime_rate": 538},
    "Los Angeles":   {"violent_crime_rate": 749},
    "Chicago":       {"violent_crime_rate": 1007},
    "Houston":       {"violent_crime_rate": 1124},
    "Miami":         {"violent_crime_rate": 629},
}
def fetch_city_crime_data(city_row, use_cache=True):
    return CRIME_CACHE[city_row["city"]]

# --- Step 3: Energy Data (EIA & City Reports, 2022) ---
# Per capita residential electricity use, kWh/year. Sourced from EIA and city energy reports.
ENERGY_CACHE = {
    "New York":      {"per_capita_kwh": 5597},   # https://www.eia.gov/electricity/state/newyork/
    "Los Angeles":   {"per_capita_kwh": 6000},   # https://www.eia.gov/electricity/state/california/
    "Chicago":       {"per_capita_kwh": 6700},   # https://www.eia.gov/electricity/state/illinois/
    "Houston":       {"per_capita_kwh": 13000},  # https://www.eia.gov/electricity/state/texas/
    "Miami":         {"per_capita_kwh": 11000},  # https://www.eia.gov/electricity/state/florida/
}
def fetch_city_energy_data(city_row, use_cache=True):
    return ENERGY_CACHE[city_row["city"]]

# --- Step 4: Build and Print DataFrames at Each Step ---

# Step 4a: Census DataFrame
city_rows = []
for city in CITIES:
    census = fetch_city_census_data(city, use_cache=True)
    city_rows.append({
        "city": city["city"],
        "state": city["state"],
        **census
    })
df_census = pd.DataFrame(city_rows)
print("=== Census Data ===")
print(df_census, "\n")

# Step 4b: Add Crime Data
city_rows = []
for city in CITIES:
    census = fetch_city_census_data(city, use_cache=True)
    crime = fetch_city_crime_data(city, use_cache=True)
    city_rows.append({
        "city": city["city"],
        "state": city["state"],
        **census,
        **crime
    })
df_crime = pd.DataFrame(city_rows)
print("=== Census + Crime Data ===")
print(df_crime, "\n")

# Step 4c: Add Energy Data
city_rows = []
for city in CITIES:
    census = fetch_city_census_data(city, use_cache=True)
    crime = fetch_city_crime_data(city, use_cache=True)
    energy = fetch_city_energy_data(city, use_cache=True)
    city_rows.append({
        "city": city["city"],
        "state": city["state"],
        **census,
        **crime,
        **energy
    })
df_cities = pd.DataFrame(city_rows)
print("=== Census + Crime + Energy Data ===")
print(df_cities, "\n")

=== Census Data ===
          city state  population  median_income
0     New York    NY     8335897          74694
1  Los Angeles    CA     3822238          69778
2      Chicago    IL     2665039          65781
3      Houston    TX     2302878          59408
4        Miami    FL      449514          53975 

=== Census + Crime Data ===
          city state  population  median_income  violent_crime_rate
0     New York    NY     8335897          74694                 538
1  Los Angeles    CA     3822238          69778                 749
2      Chicago    IL     2665039          65781                1007
3      Houston    TX     2302878          59408                1124
4        Miami    FL      449514          53975                 629 

=== Census + Crime + Energy Data ===
          city state  population  median_income  violent_crime_rate  \
0     New York    NY     8335897          74694                 538   
1  Los Angeles    CA     3822238          69778                 749   
2 

In [3]:
cities = [
    {"city": "New York",      "state": "NY"},
    {"city": "Los Angeles",   "state": "CA"},
    {"city": "Chicago",       "state": "IL"},
    {"city": "Houston",       "state": "TX"},
    {"city": "Miami",         "state": "FL"},
]

In [5]:
# NYC 311 Operations Analytics - Full Pipeline (Colab Ready)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
import seaborn as sns

# 1. Data Ingestion (100k+ records, last 60 days)
url = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$limit=100000&$where=created_date>'2024-05-01T00:00:00.000'"
df = pd.read_csv(url, parse_dates=['created_date', 'closed_date'], low_memory=False)
print("Raw shape:", df.shape)

# 2. Feature Engineering
df = df[['unique_key', 'created_date', 'closed_date', 'complaint_type', 'borough', 'latitude', 'longitude']]
df = df.dropna(subset=['created_date', 'complaint_type', 'borough'])
df['hour'] = df['created_date'].dt.hour
df['weekday'] = df['created_date'].dt.dayofweek
df['is_closed'] = ~df['closed_date'].isna()
df['closed_same_day'] = (df['closed_date'].dt.date == df['created_date'].dt.date) & df['is_closed']
df['closed_same_day'] = df['closed_same_day'].astype(int)

# Only keep rows with closed_date or at least 7 days old (give time to close)
df_recent = df[(df['is_closed']) | (df['created_date'] < (pd.Timestamp.now() - pd.Timedelta(days=7)))]

# Top 10 complaint types only (to make modeling tractable)
top_complaints = df_recent['complaint_type'].value_counts().head(10).index
df_recent = df_recent[df_recent['complaint_type'].isin(top_complaints)]

# 3. Prepare Features
X = pd.get_dummies(df_recent[['complaint_type', 'borough', 'hour', 'weekday']], drop_first=True)
y = df_recent['closed_same_day']

# 4. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 5. Fit Model
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')
clf.fit(X_train, y_train)

# 6. Evaluate Model
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:,1]
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))
print("ROC AUC:", roc_auc_score(y_test, y_prob))

# 7. Feature Importance
importances = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nTop 10 Feature Importances:")
print(importances.head(10))

plt.figure(figsize=(10,5))
importances.head(10).plot(kind='barh')
plt.title("Top 10 Feature Importances")
plt.gca().invert_yaxis()
plt.show()

# 8. Trend Visualization: Same-day Closure Rate by Complaint Type (last 30 days)
df_recent['date'] = df_recent['created_date'].dt.date
summary = df_recent.groupby(['date', 'complaint_type'])['closed_same_day'].mean().unstack().tail(30)

plt.figure(figsize=(12,6))
summary.plot(ax=plt.gca())
plt.title("Same-Day Closure Rate by Complaint Type (Last 30 Days)")
plt.ylabel("Closure Rate")
plt.xlabel("Date")
plt.legend(title="Complaint Type", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

KeyboardInterrupt: 

In [6]:
import plotly.express as px

# Pivot for heatmap: rows=borough, columns=hour
heatmap_df = agg.pivot_table(index='borough', columns='hour', values='closure_rate', fill_value=0)

fig = px.imshow(
    heatmap_df,
    text_auto=True,
    aspect='auto',
    color_continuous_scale='Blues',
    labels=dict(x="Hour of Day", y="Borough", color="Closure Rate"),
    title="Closure Rate Heatmap by Borough and Hour"
)
fig.update_layout(height=400)
fig.show()

NameError: name 'agg' is not defined

In [7]:
def scenario_dashboard(borough, hour, complaint_type, boost_pct):
    new_agg = simulate(agg, borough, hour, complaint_type, boost_pct/100.0)
    before = agg[(agg['borough']==borough) & (agg['hour']==hour) & (agg['complaint_type']==complaint_type)]
    after = new_agg[(new_agg['borough']==borough) & (new_agg['hour']==hour) & (new_agg['complaint_type']==complaint_type)]
    if before.empty or after.empty:
        print("No data for this combination.")
        return

    before = before.iloc[0]
    after = after.iloc[0]
    print(f"Before: Closure Rate: {before['closure_rate']:.1%}, Expected Cost: ${before['expected_cost']:,.0f}")
    print(f"After:  Closure Rate: {after['closure_rate']:.1%}, Expected Cost: ${after['expected_cost']:,.0f}")
    print(f"Savings: ${before['expected_cost']-after['expected_cost']:,.0f} per period")
    print("\nRecommendation: Prioritize high-cost complaint types and low-performing boroughs at key hours.")

    # Plot Top 10 Costliest
    top_cost = agg.sort_values('expected_cost', ascending=False).head(10)
    fig = px.bar(top_cost, x='expected_cost', y='complaint_type', color='borough', orientation='h',
                 labels={'expected_cost':'Expected Unresolved Cost', 'complaint_type':'Complaint Type'})
    fig.show()

    # --- Add the Before/After Bar Chart here ---
    import plotly.graph_objects as go
    labels = ['Before', 'After']
    values = [before['expected_cost'], after['expected_cost']]
    fig2 = go.Figure(go.Bar(x=labels, y=values, marker_color=['#636EFA', '#EF553B']))
    fig2.update_layout(
        title=f'Expected Unresolved Cost: {complaint_type} in {borough} at {hour}:00',
        yaxis_title='Expected Cost ($)'
    )
    fig2.show()
    # -------------------------------------------

    # Closure Rate Heatmap
    heat_df = agg.pivot_table(index='hour', columns='borough', values='closure_rate', aggfunc='mean')
    fig3 = px.imshow(heat_df, aspect='auto', color_continuous_scale='Blues',
                     labels=dict(color="Closure Rate"), title="Closure Rate by Hour and Borough")
    fig3.show()

In [8]:
# Install ipywidgets if not present
!pip install ipywidgets plotly --quiet

import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# --- 1. Data Ingestion ---
URL = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$limit=80000&$where=created_date>'2024-06-01T00:00:00.000'"
df = pd.read_csv(URL, parse_dates=['created_date', 'closed_date'], low_memory=False)

df = df[['unique_key', 'created_date', 'closed_date', 'complaint_type', 'borough', 'latitude', 'longitude']]
df = df.dropna(subset=['created_date', 'complaint_type', 'borough'])

# --- 2. Feature Engineering ---
df['hour'] = df['created_date'].dt.hour
df['weekday'] = df['created_date'].dt.dayofweek
df['is_closed'] = ~df['closed_date'].isna()
df['closed_same_day'] = (df['closed_date'].dt.date == df['created_date'].dt.date) & df['is_closed']
df['closed_same_day'] = df['closed_same_day'].astype(int)
df['date'] = df['created_date'].dt.date

# Only top 10 complaint types
top_types = df['complaint_type'].value_counts().head(10).index
df = df[df['complaint_type'].isin(top_types)]

# --- 3. Assign Cost/Priority Scores ---
cost_map = {
    'Dirty Condition': 400,
    'Water System': 1000,
    'Street Condition': 700,
    'Illegal Parking': 150,
    'Drug Activity': 1200,
    'Noise - Residential': 200,
    'Abandoned Vehicle': 300,
    'Blocked Driveway': 100,
    'Noise - Street/Sidewalk': 200,
    'Encampment': 1600
}
df['cost_per_unresolved'] = df['complaint_type'].map(cost_map).fillna(200)

# --- 4. Aggregate: Closure Rate & Cost by Borough/Hour/Type ---
agg = df.groupby(['borough', 'hour', 'complaint_type']).agg(
    total=('unique_key','count'),
    closed_same_day=('closed_same_day','sum'),
    avg_cost=('cost_per_unresolved','mean')
).reset_index()
agg['closure_rate'] = agg['closed_same_day'] / agg['total']
agg['expected_cost'] = (agg['total'] - agg['closed_same_day']) * agg['avg_cost']

# --- 5. Scenario Simulation ---
def simulate(agg_df, borough, hour, complaint_type, boost_pct):
    df_sim = agg_df.copy()
    mask = (df_sim['borough']==borough) & (df_sim['hour']==hour) & (df_sim['complaint_type']==complaint_type)
    if mask.sum() == 0:
        return df_sim
    old_rate = df_sim.loc[mask, 'closure_rate'].values[0]
    # Cap at 99%
    new_rate = min(0.99, old_rate + boost_pct*0.3)
    df_sim.loc[mask, 'closure_rate'] = new_rate
    df_sim.loc[mask, 'closed_same_day'] = (df_sim.loc[mask, 'total'] * new_rate).astype(int)
    df_sim.loc[mask, 'expected_cost'] = (df_sim.loc[mask, 'total'] - df_sim.loc[mask, 'closed_same_day']) * df_sim.loc[mask, 'avg_cost']
    return df_sim

# --- 6. Widget UI ---
borough_widget = widgets.Dropdown(options=agg['borough'].unique(), description='Borough:')
hour_widget = widgets.IntSlider(min=0, max=23, step=1, value=8, description='Hour:')
complaint_widget = widgets.Dropdown(options=agg['complaint_type'].unique(), description='Complaint:')
boost_widget = widgets.IntSlider(min=0, max=100, step=5, value=20, description='Boost %:')

def scenario_dashboard(borough, hour, complaint_type, boost_pct):
    new_agg = simulate(agg, borough, hour, complaint_type, boost_pct/100.0)
    before = agg[(agg['borough']==borough) & (agg['hour']==hour) & (agg['complaint_type']==complaint_type)]
    after = new_agg[(new_agg['borough']==borough) & (new_agg['hour']==hour) & (new_agg['complaint_type']==complaint_type)]
    if before.empty or after.empty:
        print("No data for this combination.")
        return

    before = before.iloc[0]
    after = after.iloc[0]
    print(f"Before: Closure Rate: {before['closure_rate']:.1%}, Expected Cost: ${before['expected_cost']:,.0f}")
    print(f"After:  Closure Rate: {after['closure_rate']:.1%}, Expected Cost: ${after['expected_cost']:,.0f}")
    print(f"Savings: ${before['expected_cost']-after['expected_cost']:,.0f} per period")
    print("\nRecommendation: Prioritize high-cost complaint types and low-performing boroughs at key hours.")

    # Plot Top 10 Costliest
    top_cost = agg.sort_values('expected_cost', ascending=False).head(10)
    fig = px.bar(top_cost, x='expected_cost', y='complaint_type', color='borough', orientation='h',
                 labels={'expected_cost':'Expected Unresolved Cost', 'complaint_type':'Complaint Type'})
    fig.show()

    # Closure Rate Heatmap
    heat_df = agg.pivot_table(index='hour', columns='borough', values='closure_rate', aggfunc='mean')
    fig2 = px.imshow(heat_df, aspect='auto', color_continuous_scale='Blues',
                     labels=dict(color="Closure Rate"), title="Closure Rate by Hour and Borough")
    fig2.show()

dashboard = widgets.interactive(
    scenario_dashboard,
    borough=borough_widget,
    hour=hour_widget,
    complaint_type=complaint_widget,
    boost_pct=boost_widget
)
display(dashboard)

interactive(children=(Dropdown(description='Borough:', options=('BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'S…

In [14]:
# --- SETUP: Install duckdb and plotly (Colab/Kaggle both OK) ---
!pip install duckdb plotly --quiet

import pandas as pd
import duckdb
import plotly.express as px

# --- LOAD DATA: NYC 311 (last 50,000 most recent) ---
url = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$order=created_date%20DESC&$limit=50000"
df = pd.read_csv(url, parse_dates=["created_date", "closed_date"], low_memory=False)

# --- PREP DATA ---
df['hour'] = df['created_date'].dt.hour
df['weekday'] = df['created_date'].dt.weekday
df['date'] = df['created_date'].dt.date
df['is_closed'] = ~df['closed_date'].isna()
df['closed_same_day'] = (
    (df['is_closed']) &
    (df['created_date'].dt.date == df['closed_date'].dt.date)
).astype(int)

# Filter to most common complaint types (for clarity)
top_types = df['complaint_type'].value_counts().head(8).index.tolist()
df = df[df['complaint_type'].isin(top_types)]

# --- DUCKDB: Calculate Demand, Backlog, Closure Rate ---
con = duckdb.connect()
con.register('df', df)

print("df columns:", df.columns)
print("Top 5 complaint types:", df['complaint_type'].value_counts().head(5))
print("Boroughs:", df['borough'].unique())
print("Sample rows:")
print(df.head())
print("Nulls per column:\n", df.isnull().sum())

query = """
SELECT
    complaint_type,
    borough,
    hour,
    COUNT(*) AS demand,
    SUM(CASE WHEN is_closed = False THEN 1 ELSE 0 END) AS backlog,
    ROUND(100.0 * SUM(closed_same_day) / COUNT(*), 1) AS pct_closed_same_day
FROM df
GROUP BY complaint_type, borough, hour
HAVING demand > 30
"""
results = con.execute(query).df()

# --- RANKINGS: Demand Rank, Backlog Rank, Priority Score ---
results['demand_rank'] = results['demand'].rank(method='min', ascending=False).astype(int)
results['backlog_rank'] = results['backlog'].rank(method='min', ascending=False).astype(int)
results['priority_score'] = results['demand_rank'] + results['backlog_rank']

# --- TOP 10 OPPORTUNITIES ---
top = results.sort_values(['priority_score', 'backlog'], ascending=[True, False]).head(10)

print("=== Top 10 Opportunities for Resource Allocation ===")
display(top[['complaint_type', 'borough', 'hour', 'demand', 'backlog', 'pct_closed_same_day', 'priority_score']])

# --- PLOT: Complaint Demand vs. Backlog (Color = Closure Rate) ---
fig = px.scatter(
    results,
    x='demand', y='backlog',
    color='pct_closed_same_day',
    hover_data=['complaint_type', 'borough', 'hour'],
    size='priority_score',
    color_continuous_scale='RdYlGn',
    title='NYC 311 Complaint Demand vs. Backlog (by Type/Borough/Hour)'
)
fig.show()

# --- PLOT: Top 10 Priority Table ---
fig2 = px.bar(
    top,
    y='complaint_type',
    x='priority_score',
    color='borough',
    orientation='h',
    text='priority_score',
    title='Top 10 Priority Complaint/Borough/Hour Combos'
)
fig2.update_layout(yaxis={'categoryorder':'total ascending'})
fig2.show()

df columns: Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'address_type',
       'city', 'landmark', 'facility_type', 'status', 'due_date',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'bbl', 'borough', 'x_coordinate_state_plane',
       'y_coordinate_state_plane', 'open_data_channel_type',
       'park_facility_name', 'park_borough', 'vehicle_type',
       'taxi_company_borough', 'taxi_pick_up_location', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'latitude', 'longitude', 'location', 'hour', 'weekday', 'date',
       'is_closed', 'closed_same_day'],
      dtype='object')
Top 5 complaint types: complaint_type
Illegal Parking            9148
Noise

,complaint_type,borough,hour,demand,backlog,pct_closed_same_day,priority_score
135,Illegal Parking,QUEENS,0,166,29.0,80.1,20
143,Illegal Parking,BROOKLYN,22,188,16.0,49.5,32
28,Illegal Parking,QUEENS,21,200,14.0,47.0,35
144,Illegal Parking,QUEENS,22,186,15.0,25.3,39
145,Noise - Street/Sidewalk,BRONX,22,190,13.0,44.7,45
29,Noise - Street/Sidewalk,BRONX,21,196,11.0,68.4,49
133,Noise - Street/Sidewalk,BRONX,0,161,13.0,91.9,57
62,Water System,BRONX,18,95,41.0,54.7,60
32,Illegal Parking,BROOKLYN,21,154,13.0,57.1,64
5,Illegal Parking,BROOKLYN,1,101,22.0,76.2,65


In [10]:
# Print all columns
print("=== Column Headers ===")
print(list(df.columns))

# Print data types and null counts
print("\n=== DataFrame Info ===")
print(df.info())

# Show value counts for key fields
for col in ['complaint_type', 'borough', 'agency', 'status']:
    if col in df.columns:
        print(f"\n=== Value counts for {col} ===")
        print(df[col].value_counts(dropna=False).head(10))

# Show describe for numerics and datetimes
print("\n=== Numeric/Datetime describe ===")
print(df.describe(include=[np.number, 'datetime']))

# Show a random sample of rows transposed for readability
print("\n=== Random Sample Rows ===")
print(df.sample(5).T)

=== Column Headers ===
['unique_key', 'created_date', 'closed_date', 'complaint_type', 'borough', 'latitude', 'longitude', 'hour', 'weekday', 'is_closed', 'closed_same_day', 'date', 'cost_per_unresolved']

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
Index: 46396 entries, 1 to 79998
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   unique_key           46396 non-null  int64         
 1   created_date         46396 non-null  datetime64[ns]
 2   closed_date          42759 non-null  datetime64[ns]
 3   complaint_type       46396 non-null  object        
 4   borough              46396 non-null  object        
 5   latitude             45546 non-null  float64       
 6   longitude            45546 non-null  float64       
 7   hour                 46396 non-null  int32         
 8   weekday              46396 non-null  int32         
 9   is_closed            46396 non-null